In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Классификатор выброного интенда.
Для болталки используем датесет: https://huggingface.co/datasets/BlackSamorez/2ch_b_dialogues

Классификатор: Fasttext

In [ ]:
import numpy as np
import pandas as pd
import re

import pickle

import nltk
nltk.download('stopwords')
from tqdm.notebook import tqdm

import string
import pymorphy2
from stop_words import get_stop_words

from sklearn.metrics import *
from sklearn.model_selection import train_test_split

import fasttext
import csv

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df_dial = pd.read_json('/content/drive/MyDrive/NLP_course_project/dialogues.jsonl', lines=True)
df_dial.head()

,id,dialogue
0,268332378,"[— я бы прошелся до крыши, — сказал он, встава..."
1,268332378,[\- да не было такого! я даже ни одной парочки...
2,268332378,[девушка стала следовать за хаккури и вскоре о...
3,268332378,[>сама дойдешь? в смысле не нужно ли ее поддер...
4,268332378,[вода на секунду приобрела зеленый оттенок. по...


In [ ]:
df_dial['category'] = 0
#df_dial[['spec', 'recep']] = 0
df_dial = df_dial.rename (columns= {'dialogue': 'text'})
df_dial['text'] = df_dial['text'].apply(lambda x: ''.join(x))
df_dial = df_dial[:10000]
df_dial.drop('id', axis=1, inplace=True)
df_dial.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,text,category
0,"— я бы прошелся до крыши, — сказал он, вставая...",0
1,\- да не было такого! я даже ни одной парочки ...,0
2,девушка стала следовать за хаккури и вскоре он...,0
3,>сама дойдешь? в смысле не нужно ли ее поддерж...,0
4,вода на секунду приобрела зеленый оттенок. пок...,0


In [ ]:
df_rec = pd.read_csv('/content/drive/MyDrive/NLP_course_project/all_recepies_inter.csv', sep='\t')
df_rec.head()

,Unnamed: 0,name,composition,cooking_type,Инструкции,dish_type,Дата,photo,source,composition_inter
0,0,рассольник классический с перловкой и солеными...,"[{'Перловка': 0.1, 'unit': 'стак. (200 мл)'}, ...","варка,жарка",Подготовить указанные ингредиенты для приготов...,первое,05.06.2015,photo_1000menu_1.jpg,https://1000.menu/cooking/33395-rassolnik-s-pe...,"[{'product_id': 4253, 'name_source': 'Перловая..."
1,1,Суп пюре из белокочаной капусты,"[{'Капуста белокочанная': 50.0, 'unit': 'гр'},...",варка,"Необходимые ингредиенты\r\nНарезаем лук, морко...",первое,27.06.2015,photo_1000menu_2.jpg,https://1000.menu/cooking/25399-sup-pure-iz-be...,"[{'product_id': 2286, 'name_source': 'Капуста ..."
2,2,Постные щи из квашеной капусты,"[{'Капуста квашеная': 116.7, 'unit': 'гр'}, {'...","варка,жарка,тушение","Честно признаюсь, у меня не было репы на момен...",первое,12.02.2013,photo_1000menu_3.jpg,https://1000.menu/cooking/5159-postnje-shchi,"[{'product_id': 0, 'name_source': 'Капуста ква..."
3,3,Тюря- простой суп быстро и вкусно,"[{'Квас': 0.2, 'unit': 'л'}, {'Лук репчатый': ...",сырое,"\r\nНачинаем мы приготовление тюри с того, что...",первое,02.03.2011,photo_1000menu_4.jpg,https://1000.menu/cooking/5085-turya,"[{'product_id': 0, 'name_source': 'Квас', 'uni..."
4,4,Фасолевый суп из красной фасоли,"[{'Вода': 0.3, 'unit': 'л'}, {'Картошка': 0.3,...",варка,Подготовить ингредиенты. Для приготовления суп...,первое,28.01.2013,photo_1000menu_5.jpg,https://1000.menu/cooking/38765-fasolevyi-sup-...,"[{'product_id': 828, 'name_source': 'Вода', 'u..."


In [ ]:
df_rec['category'] = 1
#df_rec[['spec', 'recep']] = 1
df_rec = df_rec[['Инструкции', 'category']][:10000]
#df_rec = df_rec[['Инструкции', 'spec', 'recep']][:10000]
df_rec = df_rec.rename (columns= {'Инструкции': 'text'})
df_rec.head()

,text,category
0,Подготовить указанные ингредиенты для приготов...,1
1,"Необходимые ингредиенты\r\nНарезаем лук, морко...",1
2,"Честно признаюсь, у меня не было репы на момен...",1
3,"\r\nНачинаем мы приготовление тюри с того, что...",1
4,Подготовить ингредиенты. Для приготовления суп...,1


In [ ]:
df_med = pd.read_csv('/content/drive/MyDrive/NLP_course_project/medical_qa_ru_data.csv')
df_med.head()

,date,categ,theme,desc,ans,spec10
0,"8 Октября 2017, 11:55",Оториноларингология,Применение Ларипронта.,"Ларипронт 20 талеток,через каждые 2-3 часа.Оче...",Что вы им лечите? Длительность приема Ларипрон...,Отоларинголог
1,"20 Февраля 2019, 13:24",Акушерство,Беременность,"Здравствуйте, я на 7-8 неделе беременности. С ...","Здравствуйте, это может быть признаком раннего...",NaN
2,"17 Марта 2015, 18:31",Другое,гинекология,Здравствуйте месячные должны придти 23 марта в...,Выполните исследование хгч,NaN
3,"13 Января 2019, 19:38",Терапия,Занятия спорта после сдачи крови,"Завтра иду с утра сдавать кровь ТТГ, Т4СВ, Кал...","Можно.;\nЗдравствуйте , да, попейте сладкого ч...",Терапевт
4,"28 Ноября 2017, 21:58",Другое,Таблетки,Мне прописали пить Аллохол. Врач написала пить...,Препарат принимается после еды. Уточните это ...,NaN


In [ ]:
df_med['text'] = df_med['desc'] + df_med['ans']
df_med['category'] = 2
#df_med['recep'] = 0
df_med = df_med[['text', 'category']][:10000]
#df_med = df_med[['text', 'spec', 'recep']][:10000]
df_med.head()

,text,category
0,"Ларипронт 20 талеток,через каждые 2-3 часа.Оче...",2
1,"Здравствуйте, я на 7-8 неделе беременности. С ...",2
2,Здравствуйте месячные должны придти 23 марта в...,2
3,"Завтра иду с утра сдавать кровь ТТГ, Т4СВ, Кал...",2
4,Мне прописали пить Аллохол. Врач написала пить...,2


In [ ]:
df_all = df_dial.append(df_rec.append(df_med))
df_all.reset_index(inplace=True)
df_all.drop('index', axis=1, inplace=True)
df_all

,text,category
0,"— я бы прошелся до крыши, — сказал он, вставая...",0
1,\- да не было такого! я даже ни одной парочки ...,0
2,девушка стала следовать за хаккури и вскоре он...,0
3,>сама дойдешь? в смысле не нужно ли ее поддерж...,0
4,вода на секунду приобрела зеленый оттенок. пок...,0
...,...,...
29995,"Добрый вечер! Болят уши где-то далеко внутри,о...",2
29996,"Здравствуйте, обнаружила сегодня на языке сбок...",2
29997,"Здравствуйте. Обьясните, пожалуйста, результат...",2
29998,Здравствуйте. Беспокоит головная боль уже ни 1...,2


In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
stopwords = get_stop_words("ru")

words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    text = str(text)
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]

def lemmatize(words, lemmer = morph, stopwords = stopwords):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

def preprocess(text):    
    return ' '.join((lemmatize(find_words(text))))

In [ ]:
df_all['text'] = df_all['text'].apply(preprocess)

In [ ]:
train, test = train_test_split(df_all, test_size=0.15, stratify= df_all['category'])

In [ ]:
with open('train.txt', 'w') as f:
    for each_text, each_label in zip(train['text'], train['category']):
        f.writelines(f'__label__{each_label} {each_text}\n')
        
with open('test.txt', 'w') as f:
    for each_text, each_label in zip(test['text'], test['category']):
        f.writelines(f'__label__{each_label} {each_text}\n')

# Training the fastText classifier
model = fasttext.train_supervised('train.txt', autotuneValidationFile='test.txt')


# Evaluating performance on the entire test file
model.test('test.txt')    

(4500, 0.9962222222222222, 0.9962222222222222)

In [ ]:
model.save_model('/content/drive/MyDrive/NLP_course_project/classificator.model')

In [ ]:
lebel = model.predict(test['text'][13224])[0][0]
lebel

'__label__1'